
<br>
<font>
<div dir=ltr align=center>
<font color=0F5298 size=7>
Minimax <br>



____

# Game explanation and environment setup

In this Jupyter notebook, we aim to develop the AI logic for the game **Connect4**. Players take turns dropping pieces into a grid, and the first player to align four consecutive pieces vertically, horizontally, or diagonally wins. The focus is on creating an intelligent AI opponent using the minimax algorithm with alpha-beta pruning to deliver a challenging gameplay experience.

However, there’s an added twist: after either player drops a piece, there is a 12.5% chance that the entire board will rotate 90 degrees clockwise, changing the direction of gravity as well!</span>
<br>
The winning condition will be checked after applying the rotation. If both sides have winning conditions, the player who made the last move will lose.
<br>

For full clarification, the following occurs:
- Some players make their moves.
- Rotations happens. (12.5%)
- New gravity is applied to every piece on the board. (12.5%)
- The winning condition is then checked.

<br>

First, we will define some constants to make the code cleaner, more organized, and to set up the game environment.

In [1]:
# Game Constants
ROW_COUNT = 7
COLUMN_COUNT = 7
WINDOW_LENGTH = 4
EMPTY = 0

# Players and Pieces
PLAYER = 0
AI = 1
EMPTY_PIECE = 0
PLAYER_PIECE = 1
AI_PIECE = 2

# Colors (RGB values) - you can change the colors to your liking
FG_COLOR = (0, 0, 255)
BG_COLOR = (0, 0, 0)
P1_COLOR = (255, 0, 0)
P2_COLOR = (0, 255, 0)

# Pygame Constants
SQUARESIZE = 80
RADIUS = SQUARESIZE // 2 - 5
SCREEN_WIDTH = COLUMN_COUNT * SQUARESIZE
SCREEN_HEIGHT = (ROW_COUNT + 1) * SQUARESIZE

# Libraries

In [2]:
!pip install pygame

import numpy as np
import random
import pygame
import math
import time



pygame 2.6.1 (SDL 2.28.4, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Util functions

We need to initialize the game board as an empty 2D array with dimensions of ROW_COUNT by COLUMN_COUNT.

In [57]:
def create_board():
    board = np.zeros((ROW_COUNT,COLUMN_COUNT))
    return board

Find the valid columns where a piece can be dropped.

In [67]:
def is_valid_location(board, col):
    if(board[ROW_COUNT-1][col] == 0):
        return True
    else:
        return False

In [68]:
def get_valid_locations(board):
    valid = []
    for col in range(COLUMN_COUNT):
    	if is_valid_location(board, col):
    		valid.append(col)
    return valid

Find the valid row for the given column where a piece can be dropped.

In [69]:
def get_next_open_row(board, col):
    for i in range(0,ROW_COUNT):
        if board[i,col] == 0:
        	return i

Drop a piece in the specified column of the board.

In [70]:
def drop_piece(board,col, piece):
    row = get_next_open_row(board,col)
    board[row][col] = piece

Check if the specified piece has won the game.

In [79]:
def winning_move(board, piece):
	# Check horizontal locations for win
	for c in range(COLUMN_COUNT-3):
		for r in range(ROW_COUNT):
			if board[r][c] == piece and board[r][c+1] == piece and board[r][c+2] == piece and board[r][c+3] == piece:
				return True

	# Check vertical locations for win
	for c in range(COLUMN_COUNT):
		for r in range(ROW_COUNT-3):
			if board[r][c] == piece and board[r+1][c] == piece and board[r+2][c] == piece and board[r+3][c] == piece:
				return True

	# Check positively sloped diaganols
	for c in range(COLUMN_COUNT-3):
		for r in range(ROW_COUNT-3):
			if board[r][c] == piece and board[r+1][c+1] == piece and board[r+2][c+2] == piece and board[r+3][c+3] == piece:
				return True

	# Check negatively sloped diaganols
	for c in range(COLUMN_COUNT-3):
		for r in range(3, ROW_COUNT):
			if board[r][c] == piece and board[r-1][c+1] == piece and board[r-2][c+2] == piece and board[r-3][c+3] == piece:
				return True

# Scoring function and Minimax implementation

Score the current situation for the player.we divide the board into separate windows, score each window individually for the given piece, and sum the scores to obtain the total board score for that piece.

In [74]:
def evaluate_window(window, piece):
	score = 0
	opp_piece = PLAYER_PIECE
	if piece == PLAYER_PIECE:
		opp_piece = AI_PIECE

	if window.count(piece) == 4:
		score += 100
	elif window.count(piece) == 3 and window.count(EMPTY) == 1:
		score += 5
	elif window.count(piece) == 2 and window.count(EMPTY) == 2:
		score += 2

	if window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
		score -= 4

	return score

In [75]:
def score_position(board, piece):
    score = 0

    center_array = board[:, COLUMN_COUNT // 2]
    center_count = list(center_array).count(piece)
    score += center_count * 3

    def score_windows(lines):
        total = 0
        for line in lines:
            for i in range(len(line) - WINDOW_LENGTH + 1):
                window = line[i:i + WINDOW_LENGTH]
                total += evaluate_window(window, piece)
        return total

    rows = [list(board[r, :]) for r in range(ROW_COUNT)]
    cols = [list(board[:, c]) for c in range(COLUMN_COUNT)]
    diagonals = [
        [board[r + i, c + i] for i in range(WINDOW_LENGTH)]
        for r in range(ROW_COUNT - 3) for c in range(COLUMN_COUNT - 3)
    ]
    anti_diagonals = [
        [board[r + 3 - i, c + i] for i in range(WINDOW_LENGTH)]
        for r in range(ROW_COUNT - 3) for c in range(COLUMN_COUNT - 3)
    ]

    score += score_windows(rows)
    score += score_windows(cols)
    score += score_windows(diagonals)
    score += score_windows(anti_diagonals)

    return score

Implement the minimax algorithm with alpha beta pruning using the utility function provided.

In [76]:
def minimax(board, depth, alpha, beta, maximizingPlayer):
    valid_locations = get_valid_locations(board)
    is_terminal = (
        winning_move(board, PLAYER_PIECE) or 
        winning_move(board, AI_PIECE) or 
        len(valid_locations) == 0
    )

    if depth == 0 or is_terminal:
        if is_terminal:
            if winning_move(board, AI_PIECE):
                return None, float('inf')
            elif winning_move(board, PLAYER_PIECE):
                return None, float('-inf')
            else:
                return None, 0
        else:
            return None, score_position(board, AI_PIECE)

    best_column = random.choice(valid_locations)

    if maximizingPlayer:
        max_value = float('-inf')
        for col in valid_locations:
            row = get_next_open_row(board, col)
            board_copy = board.copy()
            drop_piece(board_copy, col, AI_PIECE)
            _, new_score = minimax(board_copy, depth - 1, alpha, beta, False)
            if new_score > max_value:
                max_value = new_score
                best_column = col
            alpha = max(alpha, max_value)
            if alpha >= beta:
                break
        return best_column, max_value

    else:
        min_value = float('inf')
        for col in valid_locations:
            row = get_next_open_row(board, col)
            board_copy = board.copy()
            drop_piece(board_copy, col, PLAYER_PIECE)
            _, new_score = minimax(board_copy, depth - 1, alpha, beta, True)
            if new_score < min_value:
                min_value = new_score
                best_column = col
            beta = min(beta, min_value)
            if alpha >= beta:
                break
        return best_column, min_value


# PVE

Test AI's performance.

In [77]:
pygame.init()
width = COLUMN_COUNT * SQUARESIZE
height = (ROW_COUNT + 1) * SQUARESIZE
size = (width, height)
screen = pygame.display.set_mode(size)
myfont = pygame.font.SysFont("monospace", 75)

def rotate_board(board):
    rotated_board = np.rot90(board)
    for col in range(rotated_board.shape[1]):
        column = rotated_board[:, col]
        non_empty = column[column != 0]
        empty_count = ROW_COUNT - len(non_empty)
        rotated_board[:, col] = np.concatenate((non_empty, np.zeros(empty_count)))
    return rotated_board

def draw_board(board):
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):
            pygame.draw.rect(screen, FG_COLOR, (c * SQUARESIZE, r * SQUARESIZE + SQUARESIZE, SQUARESIZE, SQUARESIZE))
            pygame.draw.circle(screen, BG_COLOR, (
                int(c * SQUARESIZE + SQUARESIZE / 2), int(r * SQUARESIZE + SQUARESIZE + SQUARESIZE / 2)), RADIUS)

    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):
            if board[r][c] == PLAYER_PIECE:
                pygame.draw.circle(screen, P1_COLOR, (
                    int(c * SQUARESIZE + SQUARESIZE / 2), height - int(r * SQUARESIZE + SQUARESIZE / 2)), RADIUS)
            elif board[r][c] == AI_PIECE:
                pygame.draw.circle(screen, P2_COLOR, (
                    int(c * SQUARESIZE + SQUARESIZE / 2), height - int(r * SQUARESIZE + SQUARESIZE / 2)), RADIUS)
    pygame.display.update()

def run_game():
    board = create_board()
    draw_board(board)
    game_over = False
    turn = random.choice([0, 1])
    while not game_over:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return

            if event.type == pygame.MOUSEMOTION:
                pygame.draw.rect(screen, BG_COLOR, (0, 0, width, SQUARESIZE))
                posx = event.pos[0]
                if turn == PLAYER:
                    pygame.draw.circle(screen, P1_COLOR, (posx, int(SQUARESIZE / 2)), RADIUS)

            pygame.display.update()

            if event.type == pygame.MOUSEBUTTONDOWN:
                pygame.draw.rect(screen, BG_COLOR, (0, 0, width, SQUARESIZE))
                if turn == PLAYER:
                    posx = event.pos[0]
                    col = int(math.floor(posx / SQUARESIZE))

                    if is_valid_location(board, col):
                        drop_piece(board,col, PLAYER_PIECE)
                        if random.random() <= 0.125:
                            board = rotate_board(board)
                            draw_board(board)
                            pygame.time.wait(1500)

                        if winning_move(board, AI_PIECE):
                            label = myfont.render("Player 2 wins!!", 1, P2_COLOR)
                            print("Player 2 wins!!")
                            screen.blit(label, (40, 10))
                            game_over = True
                        elif winning_move(board, PLAYER_PIECE):
                            label = myfont.render("Player 1 wins!!", 1, P1_COLOR)
                            print("Player 1 wins!!")
                            screen.blit(label, (40, 10))
                            game_over = True

                        turn += 1
                        turn = turn % 2
                        draw_board(board)

        if turn == AI and not game_over:
            col, minimax_score = minimax(board, 5, -math.inf, math.inf, True)

            if is_valid_location(board, col):
                drop_piece(board,col, AI_PIECE)

                if random.random() <= 0.125:
                    board = rotate_board(board)
                    draw_board(board)
                    pygame.time.wait(1500)

                if winning_move(board, PLAYER_PIECE):
                    label = myfont.render("Player 1 wins!!", 1, P1_COLOR)
                    print("Player 1 wins!!")
                    screen.blit(label, (40, 10))
                    game_over = True
                elif winning_move(board, AI_PIECE):
                    label = myfont.render("Player 2 wins!!", 1, P2_COLOR)
                    print("Player 2 wins!!")
                    screen.blit(label, (40, 10))
                    game_over = True

                turn += 1
                turn = turn % 2
                draw_board(board)

        if game_over:
            pygame.time.wait(3000)
            pygame.quit()
            return


run_game()

# EVE

In this section, we will simulate an AI battle where our AI heuristic should outperform our provided heuristic.

Implement the minimax algorithm similar to  main minimax function.

In [78]:
def tester_evaluate_window(window, piece):
    score = 0
    opp_piece = PLAYER_PIECE if piece == AI_PIECE else AI_PIECE
    if window.count(piece) == 4:
        score += 100
    elif window.count(piece) == 3 and window.count(EMPTY) == 1:
        score += 5
    elif window.count(piece) == 2 and window.count(EMPTY) == 2:
        score += 2
    if window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
        score -= 4
    return score


def tester_score_position(board, piece):
    score = 0
    center_array = [int(i) for i in list(board[:, COLUMN_COUNT // 2])]
    score += center_array.count(piece) * 3

    for r in range(ROW_COUNT):
        row_array = [int(i) for i in list(board[r, :])]
        for c in range(COLUMN_COUNT - 3):
            window = row_array[c:c + WINDOW_LENGTH]
            score += tester_evaluate_window(window, piece)

    for c in range(COLUMN_COUNT):
        col_array = [int(i) for i in list(board[:, c])]
        for r in range(ROW_COUNT - 3):
            window = col_array[r:r + WINDOW_LENGTH]
            score += tester_evaluate_window(window, piece)

    for r in range(ROW_COUNT - 3):
        for c in range(COLUMN_COUNT - 3):
            window = [board[r + i][c + i] for i in range(WINDOW_LENGTH)]
            score += tester_evaluate_window(window, piece)

    for r in range(ROW_COUNT - 3):
        for c in range(COLUMN_COUNT - 3):
            window = [board[r + 3 - i][c + i] for i in range(WINDOW_LENGTH)]
            score += tester_evaluate_window(window, piece)


def tester_minimax(board, depth, alpha, beta, maximizingPlayer):
    valid_locations = get_valid_locations(board)
    is_terminal = (
        winning_move(board, PLAYER_PIECE) or 
        winning_move(board, AI_PIECE) or 
        len(valid_locations) == 0
    )

    if depth == 0 or is_terminal:
        if is_terminal:
            if winning_move(board, AI_PIECE):
                return None, float('inf')
            elif winning_move(board, PLAYER_PIECE):
                return None, float('-inf')
            else:
                return None, 0
        else:
            return None, tester_score_position(board, AI_PIECE)

    best_column = random.choice(valid_locations)

    if maximizingPlayer:
        max_value = float('-inf')
        for col in valid_locations:
            row = get_next_open_row(board, col)
            board_copy = board.copy()
            drop_piece(board_copy, col, AI_PIECE)
            _, new_score = minimax(board_copy, depth - 1, alpha, beta, False)
            if new_score > max_value:
                max_value = new_score
                best_column = col
            alpha = max(alpha, max_value)
            if alpha >= beta:
                break
        return best_column, max_value

    else:
        min_value = float('inf')
        for col in valid_locations:
            row = get_next_open_row(board, col)
            board_copy = board.copy()
            drop_piece(board_copy, col, PLAYER_PIECE)
            _, new_score = minimax(board_copy, depth - 1, alpha, beta, True)
            if new_score < min_value:
                min_value = new_score
                best_column = col
            beta = min(beta, min_value)
            if alpha >= beta:
                break
        return best_column, min_value

In [72]:
def simulate_game():
    board = create_board()
    starting_turn = turn = random.choice([0, 1])

    game_over = False
    while not game_over:
        if turn == PLAYER:
            col, minimax_score = tester_minimax(board, 4, -math.inf, math.inf, True)
            if is_valid_location(board, col):
                drop_piece(board, col, PLAYER_PIECE)
                if random.random() <= 0.125:
                    board = rotate_board(board)

                if winning_move(board, AI_PIECE):
                    return 1, starting_turn

                elif winning_move(board, PLAYER_PIECE):
                    return 0, starting_turn
                turn += 1
                turn = turn % 2

        if turn == AI and not game_over:
            col, minimax_score = minimax(board, 4, -math.inf, math.inf, True)
            if is_valid_location(board, col):
                drop_piece(board, col, AI_PIECE)
                if random.random() <= 0.125:
                    board = rotate_board(board)
                if winning_move(board, PLAYER_PIECE):
                    return 0, starting_turn
                elif winning_move(board, AI_PIECE):
                    return 1, starting_turn

                turn += 1
                turn = turn % 2

The code execution should take less than 10 minutes to complete.

In [81]:
wins = 0
tests = 100
throw = 0
disadvantage = 0

start_time = time.time()

for i in range(tests):
    result = simulate_game()
    if result[1] == PLAYER and result[0] == 0: disadvantage += 1
    if result[1] == AI and result[0] == 0: throw += 1
    wins += result[0]

end_time = time.time()

print(f"you won {wins / tests}% of games")
print(f"you throw {throw / tests}% of games")
print(f"you lost logically {disadvantage / tests}% of games")
print(f"Code execution time: {end_time - start_time:.4f} seconds")

you won 0.73% of games
you throw 0.1% of games
you lost logically 0.17% of games
Code execution time: 132.0796 seconds
